In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Ali\Documents\python\stack-overflow-developer-survey-2025\survey_results_public.csv")
schema = pd.read_csv(r"C:\Users\Ali\Documents\python\stack-overflow-developer-survey-2025\survey_results_schema.csv")

In [3]:
df.head()

,ResponseId,MainBranch,Age,EdLevel,Employment,EmploymentAddl,WorkExp,LearnCodeChoose,LearnCode,LearnCodeAI,...,AIAgentOrchestration,AIAgentOrchWrite,AIAgentObserveSecure,AIAgentObsWrite,AIAgentExternal,AIAgentExtWrite,AIHuman,AIOpen,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,25-34 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Employed,"Caring for dependents (children, elderly, etc.)",8.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,Vertex AI,NaN,NaN,NaN,ChatGPT,NaN,When I don’t trust AI’s answers,"Troubleshooting, profiling, debugging",61256.0,10.0
1,2,I am a developer by profession,25-34 years old,"Associate degree (A.A., A.S., etc.)",Employed,NaN,2.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers;When I want to...,All skills. AI is a flop.,104413.0,9.0
2,3,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Independent contractor, freelancer, or self-em...",None of the above,10.0,"Yes, I am not new to coding but am learning ne...",Online Courses or Certification (includes all ...,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code;GitHub Copilot;Google Gemini,NaN,When I don’t trust AI’s answers;When I want to...,"Understand how things actually work, problem s...",53061.0,8.0
3,4,I am a developer by profession,35-44 years old,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Employed,None of the above,4.0,"Yes, I am not new to coding but am learning ne...","Other online resources (e.g. standard search, ...","Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,ChatGPT;Claude Code,NaN,When I don’t trust AI’s answers;When I want to...,NaN,36197.0,6.0
4,5,I am a developer by profession,35-44 years old,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Independent contractor, freelancer, or self-em...","Caring for dependents (children, elderly, etc.)",21.0,"No, I am not new to coding and did not learn n...",NaN,"Yes, I learned how to use AI-enabled tools for...",...,NaN,NaN,NaN,NaN,NaN,NaN,When I don’t trust AI’s answers,"critical thinking, the skill to define the tas...",60000.0,7.0


In [4]:
schema.head()

,qid,qname,question,type,sub,sq_id
0,QID18,TechEndorse_1,What attracts you to a technology or causes yo...,RO,AI integration or AI Agent capabilities,1.0
1,QID18,TechEndorse_2,What attracts you to a technology or causes yo...,RO,Easy-to-use API,2.0
2,QID18,TechEndorse_3,What attracts you to a technology or causes yo...,RO,Robust and complete API,3.0
3,QID18,TechEndorse_4,What attracts you to a technology or causes yo...,RO,Customizable and manageable codebase,4.0
4,QID18,TechEndorse_5,What attracts you to a technology or causes yo...,RO,Reputation for quality,5.0


In [5]:
df.shape

(49123, 170)

In [6]:
schema.shape

(139, 6)

In [7]:
df.columns

Index(['ResponseId', 'MainBranch', 'Age', 'EdLevel', 'Employment',
       'EmploymentAddl', 'WorkExp', 'LearnCodeChoose', 'LearnCode',
       'LearnCodeAI',
       ...
       'AIAgentOrchestration', 'AIAgentOrchWrite', 'AIAgentObserveSecure',
       'AIAgentObsWrite', 'AIAgentExternal', 'AIAgentExtWrite', 'AIHuman',
       'AIOpen', 'ConvertedCompYearly', 'JobSat'],
      dtype='object', length=170)

In [8]:
schema.columns

Index(['qid', 'qname', 'question', 'type', 'sub', 'sq_id'], dtype='object')

### 1-How many respondents completed the survey?

In [10]:
df.ResponseId.nunique()

49123

### 2-How many respondents answered all questions?

In [12]:
questions = set(schema.qname.unique()) & set(df.columns)
df.dropna(subset=questions).shape[0]

0

In [13]:
# map qid -> the qname columns that exist in df
qid_to_cols = (schema[schema.qname.isin(df.columns)]
               .groupby('qid')['qname'].apply(list))

# build a boolean frame: True if respondent answered at least one col for that question group
answered_any = pd.DataFrame({
    qid: df[cols].notna().any(axis=1) for qid, cols in qid_to_cols.items()
})

# --- Option 1: strict (all question groups answered)
answered_all = answered_any.all(axis=1).sum()

# --- Option 2: core only (asked to ≥80% of respondents)
core_qids = answered_any.mean(axis=0).loc[lambda s: s >= 0.80].index
answered_all_core = answered_any[core_qids].all(axis=1).sum()

print("All question groups answered:", answered_all)
print("All *core* question groups answered:", answered_all_core)

All question groups answered: 1
All *core* question groups answered: 37131


All question groups answered: 1 → only one person filled something for every single qid (including rare/branch questions). That’s expected, since many questions are optional or only shown to subgroups.

All *core* question groups answered: 37,131 → a much more meaningful number: these are respondents who answered all the questions that were asked to ≥80% of people. This is the metric you should use for analysis, because it excludes write-ins and branch-only questions.

### 3-What are the values ​​of measures of central tendency for respondents' experience (WorkExp)?

In [16]:
print(f'Mean: {round(df.WorkExp.mean(), 1)}')
print(f'Mode: {round(df.WorkExp.mode()[0], 1)}')
print(f'Median: {round(df.WorkExp.median(), 1)}')

Mean: 13.4
Mode: 10.0
Median: 10.0


### 4-How many respondents work remotely?

In [18]:
df[df.RemoteWork == 'Remote'].shape[0]

10924

### 5-What percentage of respondents program in Python?

In [20]:
# First option
df['worked_with_python'] = df.LanguageHaveWorkedWith.apply(lambda x: 1 if 'python' in str(x).lower() else 0)
round(df.worked_with_python.sum() / df.ResponseId.nunique(), 2)

0.38

In [21]:
# Second option
df['worked_with_python'] = df.LanguageHaveWorkedWith.str.contains('python', case=False, na=False)
round(df.worked_with_python.sum() / df.ResponseId.nunique(), 2)

0.38

### 6-How many respondents learned to program using online courses?

In [23]:
# First option
df['learned_with_online_courses'] = df.LearnCode.apply(lambda x: 1 if 'online courses' in str(x).lower() else 0)
df.learned_with_online_courses.sum()

10957

In [24]:
# Second option
df['learned_with_online_courses'] = df.LearnCode.str.contains('online courses', case=False, na=False)
df.learned_with_online_courses.sum()

10957

### 7-Among the respondents who program in Python, grouped by country, what is the average and median amount of compensation (ConvertedCompYearly) in each country?

In [26]:
py_progers_by_country = df[df.worked_with_python == 1].dropna(subset='ConvertedCompYearly') \
                                                      .groupby('Country', as_index=False) \
                                                      .agg({'ConvertedCompYearly': ['mean', 'median']}).droplevel(1, axis=1)
py_progers_by_country.columns = ['country', 'mean_comp', 'median_comp']
py_progers_by_country = py_progers_by_country.astype({'mean_comp': 'int', 'median_comp': 'int'})
py_progers_by_country

,country,mean_comp,median_comp
0,Afghanistan,22328,1000
1,Albania,47217,50000
2,Algeria,20187,7088
3,Andorra,226103,226103
4,Antigua and Barbuda,1,1
...,...,...,...
148,"Venezuela, Bolivarian Republic of...",9908,3000
149,Viet Nam,218837,8254
150,Yemen,32929,23672
151,Zambia,5424,3206


### 8-What are the education levels of the 5 respondents with the highest compensation?

In [28]:
df[['ResponseId', 'EdLevel', 'ConvertedCompYearly']].sort_values(by='ConvertedCompYearly', ascending=False).reset_index(drop=True).head(5)

,ResponseId,EdLevel,ConvertedCompYearly
0,34268,"Associate degree (A.A., A.S., etc.)",50000000.0
1,28701,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",33552715.0
2,43144,"Associate degree (A.A., A.S., etc.)",18387548.0
3,35354,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",15430267.0
4,45972,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",13921760.0


### bonus1-In each age category, what percentage of respondents program in Python?

In [30]:
py_progers_share_by_age = df.groupby('Age', as_index=False).agg({'ResponseId': 'count', 'worked_with_python': 'sum'})
py_progers_share_by_age['work_with_python_share'] = round(py_progers_share_by_age.worked_with_python / py_progers_share_by_age.ResponseId, 2)
py_progers_share_by_age

,Age,ResponseId,worked_with_python,work_with_python_share
0,18-24 years old,9195,3681,0.40
1,25-34 years old,16485,6090,0.37
2,35-44 years old,13232,4857,0.37
3,45-54 years old,6265,2422,0.39
4,55-64 years old,2626,978,0.37
5,65 years or older,942,298,0.32
6,Prefer not to say,378,118,0.31


### bonus2-Among respondents who are in the 75th percentile for average compensation and who work remotely, which industries are the most common?

In [32]:
df[(df.ConvertedCompYearly > df.ConvertedCompYearly.quantile(0.75)) & (df.RemoteWork == 'Remote')].Industry.value_counts().reset_index()

,Industry,count
0,Software Development,1184
1,Fintech,189
2,Healthcare,188
3,Other:,176
4,"Internet, Telecomm or Information Services",138
5,Banking/Financial Services,88
6,Government,78
7,Media & Advertising Services,75
8,Retail and Consumer Services,64
9,"Transportation, or Supply Chain",63
